Used for ceating a cleanset of Monolingual words to used for sanitization of predictions.

Data can be of text corpus or word list, and script cleans words with diffenrent language unicodes.
Objective is distillas much as possible to purest words yest maintaining a larger word set 

In [ ]:
!wget  data.zip


## Reading Files 

In [ ]:
##Directly from TXT
with open("/content/monolingual/monolingual.hi") as f:
    raw_data = f.read()

### For AI4Bharat word frequency

In [ ]:
with open("/content/hi.vocabfreq.tsv") as f:
    data = f.readlines()

## Take only Frequency greater than 1
import re
word_list = []
for d in data:
    y = re.split("\t|\n", d)
    if int(y[1]) > 1:
        word_list.append(y[0]) 

## Cleaning the text 

In [ ]:
devanagari_scripts = {'ऄ','अ', 'आ', 'इ', 'ई', 'उ', 'ऊ','ऍ', 'ऎ', 'ए', 'ऐ',
    'ऑ', 'ऒ', 'ओ', 'औ','ऋ','ॠ','ऌ','ॡ','ॲ', 'ॐ',
    'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण',
    'त', 'थ', 'द', 'ध', 'न', 'ऩ', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ऱ', 'ल',
    'ळ', 'ऴ', 'व', 'श', 'ष', 'स', 'ह', 'क़', 'ख़', 'ग़', 'ज़', 'ड़', 'ढ़', 'फ़', 'य़',
    '्', 'ा', 'ि', 'ी', 'ु', 'ू', 'ॅ', 'ॆ', 'े', 'ै', 'ॉ', 'ॊ', 'ो', 'ौ',
    'ृ', 'ॄ', 'ॢ', 'ॣ', 'ँ', 'ं', 'ः', '़', '॑',  'ऽ',

    chr(0x200c), # ZeroWidth-NonJoiner U+200c
    chr(0x200d), # ZeroWidthJoiner U+200d
}

In [ ]:
## Removal Set

Spl_puncs = [chr(i) for i in range(0x2000,0x208F) ]
Latin_chars = [chr(i) for i in range(0x0020,0x007F)]
Latin_supli_exten = [chr(i) for i in range(0x00A1,0x017F)]
dgri_num = [chr(i) for i in range(0x0966, 0x0970)]
Other_chars = ["\n",'×', "॥", '।', '॰']

remove_chars_set = Latin_chars + Spl_puncs + Latin_supli_exten + dgri_num + Other_chars 
print(remove_chars_set)

In [ ]:
clean_data = raw_data
for c in remove_chars_set:
    clean_data = clean_data.replace(c, " ")

word_tks = clean_data.split()

In [ ]:
#print(word_tks) 
print(len(word_tks))

In [ ]:
word_dict = dict()
for w in word_tks:
    v = word_dict.get(w)
    if v:
        word_dict[w] = v+1
    else:
        word_dict[w] = 1



In [ ]:
# print(word_dict)
print(len(word_dict))

In [ ]:
## Check if the characters are devanagari
new_set = set(word_dict)
removal_word_set = set()
for n in new_set:
    for c in set(n):
        if c in devanagari_scripts:
            continue
        else:
            print(n)
            removal_word_set.add(n)
            break
print("Removal set", len(removal_word_set))

In [ ]:
for k in removal_word_set:
    word_dict.pop(k)

## Save data

In [ ]:
import pandas as pd
import csv
data_items = word_dict.items()
# data_list = list(data_items)
df = pd.DataFrame(data_items, columns =["WORD", "FREQ"] )
df = df.sort_values(["FREQ"], ascending= False, ignore_index =True)

df.to_csv('out_data.csv', index=False, quoting=csv.QUOTE_NONE)

In [ ]:
word_list = sorted(list(word_dict))

import json
with open("hi_words.json" ,"w", encoding = "utf-8") as f:
    json.dump(data_dict, f, ensure_ascii=False, indent=4, sort_keys=True,)